In [ ]:
# generic to specifiy if gpu or cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# loading the model
model = models.resnet50(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.Dropout(0.25),
               nn.ReLU(inplace=True),
               nn.Linear(128, 4)).to(device)